
**Importing Libraries**

In [1]:
import pandas as pd
import ast
from datetime import datetime
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'iframe_connected'



****Loading dataset****

In [2]:
metadata_path = "/kaggle/input/nasa-battery-dataset/cleaned_dataset/metadata.csv"
metadata_df = pd.read_csv(metadata_path)

**column headers**

In [3]:
print(metadata_df.columns)

Index(['type', 'start_time', 'ambient_temperature', 'battery_id', 'test_id',
       'uid', 'filename', 'Capacity', 'Re', 'Rct'],
      dtype='object')


**Filtering out dataset with type impedance**

In [4]:
impedance_df = metadata_df[metadata_df['type'] == 'impedance'][['start_time', 'Re', 'Rct', 'battery_id']]

**creating original dataframe of dataset**

In [5]:
dataset_df = metadata_df[metadata_df['type'] == 'impedance'][['start_time', 'filename', 'battery_id']]

**start time to appropriate format**

In [6]:
from datetime import datetime

def parse_start_time(value):
    try:
        if isinstance(value, str):
            value = value.strip("[]").replace(",", "")
            components = [float(x) for x in value.split()]
            if len(components) == 6:
                year, month, day, hour, minute = map(int, components[:5])
                second = int(components[5])
                return datetime(year, month, day, hour, minute, second)
    except (ValueError, SyntaxError, TypeError):
        return pd.NaT
    return pd.NaT


dataset_df['start_time'] = dataset_df['start_time'].apply(parse_start_time)
impedance_df['start_time'] = impedance_df['start_time'].apply(parse_start_time)

**Dropping null values and sorting**

In [7]:
impedance_df = impedance_df.dropna(subset=['start_time'])
impedance_df = impedance_df.sort_values(by='start_time')

dataset_df = dataset_df.dropna(subset=['start_time'])
dataset_df = dataset_df.sort_values(by='start_time')

**list of batteries**

In [8]:
print(impedance_df['battery_id'].unique())

['B0005' 'B0007' 'B0006' 'B0018' 'B0028' 'B0027' 'B0025' 'B0026' 'B0029'
 'B0031' 'B0030' 'B0032' 'B0036' 'B0034' 'B0033' 'B0039' 'B0038' 'B0040'
 'B0043' 'B0044' 'B0042' 'B0041' 'B0047' 'B0045' 'B0046' 'B0048' 'B0052'
 'B0049' 'B0050' 'B0051' 'B0054' 'B0056' 'B0055' 'B0053']


**Filtering out information for each battery_id and plotting curves**

In [9]:
for battery_id in impedance_df['battery_id'].unique() :
    current_battery_info = impedance_df[impedance_df['battery_id'] == battery_id]
    # print(current_battery_info)
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=current_battery_info['start_time'], 
        y=current_battery_info['Re'], 
        mode='lines', 
        name='Re',
        line=dict(color='green')
    ))
    fig.add_trace(go.Scatter(
        x=current_battery_info['start_time'], 
        y=current_battery_info['Rct'], 
        mode='lines', 
        name='Rct',
        line=dict(color='blue')
    ))
    fig.update_layout(
        title=f"Resistance (Battery ID: {battery_id})",
        xaxis_title="Start Time",
        yaxis_title="Resistance (Ohms)",
        xaxis=dict(tickangle=45),
        legend_title="Legend",
        template="plotly"
    )
    fig.show()

**Analyzing Battery Impedance Over Time**

In [10]:
battery_info = {}
for idx, row in dataset_df.iterrows() :
    battery_id = row['battery_id']
    filename = row['filename']
    start_time = row['start_time']
    filePath = f"/kaggle/input/nasa-battery-dataset/cleaned_dataset/data/{filename}"
    try:
        data = pd.read_csv(filePath)
        
        if 'Battery_impedance' in data.columns:
            impedance_values = data['Battery_impedance'].apply(lambda x: complex(x).real)
            avg_impedance = impedance_values.mean()
            if battery_id not in battery_info:
                battery_info[battery_id] = {'start_time': [], 'battery_impedance': []}
            battery_info[battery_id]['start_time'].append(start_time)
            battery_info[battery_id]['battery_impedance'].append(avg_impedance)
        else:
            print(f"'Battery_impedance' column not found in {filename}")
    except Exception as e:
        print(f"Error occurred with {filename}: {e}")
for battery_id, data in battery_info.items():
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=data['start_time'],
                                 y=data['battery_impedance'],
                                 mode='lines',
                                 name=f'Battery {battery_id}')
                     )
        fig.update_layout(
            title=f'Battery impedance change over time for battery {battery_id}',
            xaxis_title='Start Time',
            yaxis_title='Battery Impedance (Ohms)',
            xaxis=dict(tickangle=45),
            template='plotly'
        )
        fig.show()